In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def createFeatures(group, i, day_ID=None):
    feat = {
        'Time Period': group.iloc[i]['Time Period'],
        'Value': group.iloc[i]['Value']
    }
    if day_ID is not None:
        feat['Day_ID'] = day_ID
    if i == 0:
        feat.update({
            'MEAN': 0,
            'STD': 0,
            'MIN': 0,
            'MAX': 0,
            'PREV': 0,
            'NUM': 0,
            'PREV_2': 0,
            'PREV_3': 0,
            'CHANGE': 0,
            'TREND': 0
        })
    else:
        prev_values = group.iloc[:i]['Value'].values
        feat.update({
            'MEAN': np.mean(prev_values),
            'STD': np.std(prev_values) if len(prev_values) > 1 else 0,
            'MIN': np.min(prev_values),
            'MAX': np.max(prev_values),
            'PREV': prev_values[-1],
            'NUM': len(prev_values),
            'PREV_2': prev_values[-2] if len(prev_values) >= 2 else 0,
            'PREV_3': prev_values[-3] if len(prev_values) >= 3 else 0,
            'CHANGE': prev_values[-1] - prev_values[-2] if len(prev_values) >= 2 else 0,
            'TREND': np.mean(np.diff(prev_values[-5:])) if len(prev_values) >= 5 else 0
        })
    return feat

In [ ]:
def extractData(group):
    features = []
    for i in range(len(group)):
        feat = createFeatures(group, i)
        features.append(feat)
    return pd.DataFrame(features)

In [ ]:
filePair = [
    ('A', 1), ('A', 2), ('A', 3), ('A', 4),
    ('B', 1), ('B', 2), ('B', 3), ('B', 4), ('B', 5),
    ('C', 1), ('C', 2), ('C', 3), ('C', 4),
    ('D', 1), ('D', 2), ('D', 3), ('D', 4), ('D', 5)
]

In [ ]:
output_file = "result.xlsx"
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

    for letter, number in filePair:
        train_file = f"{letter}_{number}_train.xlsx"
        test_file = f"{letter}_{number}_test.xlsx"
        sheet_name = f"{letter}{number}"

        print(f"Processing {train_file} and {test_file}...")

        train = pd.read_excel(train_file)
        test = pd.read_excel(test_file)

        trainFeatures = pd.concat([extractData(group) for _, group in train.groupby('Day_ID')], ignore_index=True)
        featureColumn = ['Time Period', 'MEAN', 'STD', 'MIN', 'MAX', 'PREV', 'NUM',
                 'PREV_2', 'PREV_3', 'CHANGE', 'TREND']
        X = trainFeatures[featureColumn]
        y = trainFeatures['Value']

        model = RandomForestRegressor(
            n_estimators=90,
            max_depth=4,
            min_samples_leaf=5,
            max_features='sqrt',
            random_state=333
        )
        model.fit(X, y)

        lst = []
        for day_ID, group in test.groupby('Day_ID'):
            group = group.reset_index(drop=True)
            for i in range(len(group)):
                feat = createFeatures(group, i, day_ID=day_ID)
                X_pred = pd.DataFrame([feat])[featureColumn]
                prediction = model.predict(X_pred)[0]
                feat['Predicted'] = prediction
                lst.append(feat)

        predictions = pd.DataFrame(lst)

        plt.figure(figsize=(12, 6))
        for day in predictions["Day_ID"].unique():
            day_data = predictions[predictions["Day_ID"] == day]
            plt.plot(day_data["Time Period"], day_data["Value"],
                     alpha=0.5, marker='o', label=f"Real Data Day {day}")
            plt.plot(day_data["Time Period"], day_data["Predicted"],
                     linewidth=2, marker='s', label=f"Predicted Data Day {day}")
        plt.legend()
        plt.xlabel("Time Period")
        plt.ylabel("Value")
        plt.title(f"Predictions for {letter}_{number}")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{letter}_{number}_plot.png")
        plt.close()

        output_df = predictions[['Time Period', 'Value', 'Predicted']]
        output_df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
import zipfile
import os
from google.colab import files

zip = 'result.zip'

with zipfile.ZipFile(zip, 'w') as zipf:
    for file in os.listdir():
        if file.endswith('.png'):
            zipf.write(file)
        if file.endswith('result.xlsx'):
            zipf.write(file)

files.download(zip)